**The Data Science Method**


1.   [Problem Identification](https://medium.com/@aiden.dataminer/the-data-science-method-problem-identification-6ffcda1e5152)

2.   [Data Wrangling](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-data-collection-organization-and-definitions-d19b6ff141c4)
  * Data Collection - Collected data from wikipedia and quandl wiki price dataset. The wikipedia showed us the currect S&P 500 companies and used their ticker symbols to query quandl wiki prices.
  * Data Organization - Done using cookiecutter
  * Data Definition
  * Data Cleaning - The S&P 500 data from quandls wiki price is clean and ready for analysis use but has lost its support from Quandl community as of April 11, 2018. So we will use this dataset to setup the protfolio optimizer with proof of concept then use a different data source later for cost efficiencies.

3.   [**Exploratory Data Analysis**](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-exploratory-data-analysis-bc84d4d8d3f9)
 * Build data profile tables and plots
        - Cumulative Return
        - Annualized Return
        - Daily Return
        - Mean Daily Return
        - Standard Deviation Daily Return
        - Simple Moving Average
        - Exponential Moving Average
        - Moving Average Convergence Divergence
        - Adj. Close & Daily Return Covariance
        - Adj. Close & Daily Return Correlation
        - Sharpe Ratio
        - Skew 
        - Kurtosis
 * Explore data relationships
 * Identification and creation of features 

4.   [Pre-processing and Training Data Development](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967)
  * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   [Modeling](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b)
  * Create dummy or indicator features for categorical variable
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   [Documentation](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-documentation-c92c28bd45e6)

  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code
  * Finalize Documentation


First, loads the needed packages and modules into Python. Then loads the data into a pandas dataframe for ease of use.

In [ ]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
# import quandl
import pandas as pd 
import numpy as np
from scipy.stats import norm
import pypfopt


import datetime as dt

import dotenv
import os

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline



In [ ]:
# prints current directory
current_dir = os.getcwd()
print("Current Directory: ")
print(current_dir)

In [ ]:
# prints parent directory
project_dir = os.path.abspath(os.path.join(os.path.join(current_dir, os.pardir), os.pardir))
print("Parent Directory: ")
print(project_dir)

In [ ]:
print(os.listdir())

In [ ]:
plt.style.use('dark_background')
c = ['white', 'springgreen', 'fuchsia', 'lightcoral', 'red'] # Color
s = [24, 20, 16, 12]  # Size
w = [0.75, 1, 1.25, 1.50] # Line Width
ga = 0.10 # Grid Alpha


In [ ]:

wiki_df = pd.read_csv(project_dir + '/data/interim/'+ 'wiki_sp500_interim.csv')
slick_df = pd.read_csv(project_dir + '/data/interim/'+ 'slick_sp500_interim.csv', index_col=['#'])
yahoo_sp500_adj_close_start_date_df = pd.read_csv(project_dir + '/data/interim/'+ 'yahoo_sp500_adj_close__start_date_interim.csv', index_col=['symbol'])
sp500_adj_close_df = pd.read_csv(project_dir + '/data/interim/'+ 'yahoo_sp500_adj_close_interim.csv', index_col=['date'], parse_dates=True)
sp500_shares_outstanding_df = pd.read_csv(project_dir + '/data/interim/'+ 'yahoo_sp500_shares_outstanding_interim.csv', index_col=['symbol'])
sp500_index_adj_close_df = pd.read_csv(project_dir + '/data/interim/'+ 'yahoo_sp500_index_adj_close_interim.csv', index_col=['date'], parse_dates=True)

In [ ]:
sp500_adj_close_df.info()
type(sp500_adj_close_df.index)

In [ ]:
yahoo_sp500_adj_close_start_date_df

In [ ]:
slick_df

In [ ]:
wiki_df

In [ ]:
sp500_adj_close_df.index

In [ ]:
sp500_adj_close_df.head()

In [ ]:
sp500_adj_close_df.info()

### Plot S&P 500 Adj Close Prices in  GICS Sectors and/or Sub Sector

In [ ]:
ALL = 'SELECT ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique


In [ ]:
tickers = None

In [ ]:

def _plot_tickers(ax, df, background_tickers, focus_tickers, legend=True, title=None, ylabel=None):
    ax.set_title(title, fontsize=s[1])
    ax.set_ylabel(ylabel, fontsize=s[2])
 
    for i in range(len(focus_tickers)):
        ax.plot(df.index.values, df[focus_tickers[i]], label=focus_tickers[i], linewidth=w[3], color=c[2])
    
    if background_tickers is not None:  
        for i in focus_tickers:
            background_tickers.remove(i)
            
#         if 10 >= len(background_tickers) > 1:
        if len(background_tickers) <= 10 and len(background_tickers) > 1:
            for i in range(len(background_tickers)):
                ax.plot(df.index.values, df[background_tickers[i]], label=background_tickers[i],  linewidth=w[3], color=c[0], alpha=0.75)
        else:
            ax.plot(df.index.values, df[background_tickers], linewidth=w[3], color=c[0], alpha=0.75)
    
    if legend:
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., title='Tickers', ncol=2)
    ax.grid(alpha=ga)


def plot_ticker_list(sector, sub_sector, ticker):

    background_tickers = None
    legend = True   
    if sector == 'SELECT ALL':
        legend = False
    else:
        if ticker != 'SELECT ALL' :
            if sub_sector == 'SELECT ALL' : 
                background_tickers = unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sector"] == sector]['Symbol'])
                background_tickers.remove('SELECT ALL')   
            elif sub_sector != 'SELECT ALL' : 
                background_tickers = unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sub-Industry"] == sub_sector]['Symbol'])
                background_tickers.remove('SELECT ALL')              
              
    fig, (ax1) = plt.subplots(1, figsize=(20, 10))
    
    _plot_tickers(ax1, sp500_adj_close_df, background_tickers, tickers, legend=legend, 
                  title=f"S&P 500 Stocks\nSector: {sector} \n Sub Industry: {sub_sector} \n Tickers: {ticker}",
                    ylabel=f"Price")
    plt.xticks(fontsize=s[3])
    plt.yticks(fontsize=s[3])
    plt.show()

    
    
def select_ticker_list(ticker):

    global tickers
    
    if ticker != 'SELECT ALL':        
        tickers = [dropdown_ticker.value]
    else:
        if dropdown_sector.value != 'SELECT ALL':
            if dropdown_sub_sector.value == 'SELECT ALL':
                tickers = unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sector"] == dropdown_sector.value]['Symbol'])
                tickers.remove('SELECT ALL')
            elif dropdown_sub_sector.value != 'SELECT ALLf':
                tickers = unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sub-Industry"] == dropdown_sub_sector.value]['Symbol'])
                tickers.remove('SELECT ALL')
        else:
            tickers = unique_sorted_values_plus_ALL(wiki_df['Symbol'])
            tickers.remove('SELECT ALL')
    return tickers


def get_ticker(sub_sector):
    global tickers

    if sub_sector != 'SELECT ALL':
        dropdown_ticker.options = unique_sorted_values_plus_ALL(
            wiki_df[wiki_df["GICS Sub-Industry"] == dropdown_sub_sector.value]['Symbol']
        )
        tickers = unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sub-Industry"] == dropdown_sub_sector.value]['Symbol'])
    else:
        dropdown_ticker.options = unique_sorted_values_plus_ALL(
            wiki_df[wiki_df["GICS Sector"] == dropdown_sector.value]['Symbol']
        )
        tickers = unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sector"] == dropdown_sector.value]['Symbol'])
    tickers.remove('SELECT ALL')


def get_sub_sector(sector):   
    global tickers

    dropdown_sub_sector.options = unique_sorted_values_plus_ALL(
        wiki_df[wiki_df["GICS Sector"] == dropdown_sector.value]['GICS Sub-Industry']
    )
    get_ticker(dropdown_sub_sector.value)
    if  dropdown_sector.value != 'SELECT ALL':
        tickers = unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sector"] == dropdown_sector.value]['Symbol'])
    else:
        tickers = unique_sorted_values_plus_ALL(wiki_df['Symbol'])
    tickers.remove('SELECT ALL')


dropdown_sector = widgets.Dropdown(
    options=unique_sorted_values_plus_ALL(wiki_df['GICS Sector']),
        description='GICS Sector:'
)

dropdown_sub_sector = widgets.Dropdown(
    options=unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sector"] == dropdown_sector.value]['GICS Sub-Industry']),
    description='Sub Sector:'
)

dropdown_ticker = widgets.Dropdown(
    options=unique_sorted_values_plus_ALL(wiki_df[wiki_df["GICS Sector"] == dropdown_sector.value]['Symbol']),
    description='Ticker:'
)

btn_plot = widgets.Button(description='Plot')


i = widgets.interactive(get_sub_sector, sector=dropdown_sector)
j = widgets.interactive(get_ticker, sub_sector=dropdown_sub_sector)
k = widgets.interactive(select_ticker_list, ticker=dropdown_ticker)

output_ticker = widgets.Output()

def btn_plot_event_handler(obj):
    output_ticker.clear_output()
    with output_ticker:
        plot_ticker_list(dropdown_sector.value, dropdown_sub_sector.value, dropdown_ticker.value)


display(i)
display(j)
display(k)

display(btn_plot)

btn_plot.on_click(btn_plot_event_handler)

tickers = unique_sorted_values_plus_ALL(wiki_df['Symbol'])
tickers.remove('SELECT ALL')



In [ ]:
display(output_ticker)

#### Cumulative Returns

In [ ]:
# Cumulative Return
def cumulative_return(df, start_date=None):
    if start_date is None:
        cr_df = (df.iloc[-1] / df.iloc[0]) - 1
    else:
        cr_df = pd.DataFrame(pd.DataFrame(index=df.columns, columns=['cumulative_return']))
        cr_df.index.name = 'symbol'
        for sym in df.columns:
            cr_df.loc[sym, 'cumulative_return'] = (df.loc[:, sym].iloc[-1] / df.loc[:, sym].loc[start_date.loc[sym,:].start_date]) - 1
    return cr_df 
            

cr = cumulative_return(sp500_adj_close_df, start_date=yahoo_sp500_adj_close_start_date_df)
print(cr.sort_values(by='cumulative_return', ascending=False).head(50))
cr_highest_25 = cr.sort_values(by='cumulative_return', ascending=False).iloc[:25, :].index.values.tolist()


#### Expected Annualized Return

In [ ]:
# Annualizes Return
def annualized_return(df, start_date=None):
    if start_date is None:
        ar_df = ((df.iloc[-1] / df.iloc[0]) ** (252 / df.count())) - 1
    else:
        ar_df = pd.DataFrame(pd.DataFrame(index=df.columns, columns=['annualized_return']))
        ar_df.index.name = 'symbol'
        for sym in df.columns:
            ar_df.loc[sym, 'annualized_return'] = ((df.loc[:, sym].iloc[-1] / df.loc[:, sym].loc[start_date.loc[sym,:].start_date]) ** (252 / df[sym].dropna().shape[0])) - 1
    return ar_df * 100

            

# annualized_return = annualized_return(sp500_adj_close_df)#, start_date=yahoo_sp500_adj_close_start_date_df)
ar = annualized_return(sp500_adj_close_df, start_date=yahoo_sp500_adj_close_start_date_df)
print(ar.sort_values(by='annualized_return', ascending=False).head(50))


#### Daily Returns

In [ ]:
# Calculate daily percentage returns  
def daily_returns(df):
    return df.pct_change() * 100

dr = daily_returns(sp500_adj_close_df) 
print(dr.head())

#### Mean Daily Returns for Each Stock

In [ ]:
# Calculate individual mean returns for each stock
def daily_returns_mean(df, daily_return=True):
    if daily_return:
        return df.mean()
    return daily_returns(df).mean()
        

mdr_s = daily_returns_mean(dr, daily_return=True)
mdr_s.name = 'mean_daily_return'
mdr = mdr_s.to_frame()
print(mdr.sort_values(by='mean_daily_return', ascending=False).head(50))
mdr_highest_25 = mdr.sort_values(by='mean_daily_return', ascending=False).iloc[:25, :].index.values.tolist()

#### Standard Deviation of Daily Returns for Each Stock

In [ ]:
# Calculate individual std of daily returns for each stock
def daily_returns_std(df, daily_return=True):
    if daily_return:
        return df.std()
    return daily_returns(df).std()

sdr_s = daily_returns_std(dr, daily_return=True)
sdr_s.name = 'std_daily_return'
sdr = sdr_s.to_frame()
print('25 Highest Risk to Lower Risk')
print(sdr.sort_values(by='std_daily_return', ascending=False).head(25))
print('----------------------------------')
print('25 Lowest Risk to Higher Risk')
print(sdr.sort_values(by='std_daily_return', ascending=True).head(25))


### Plot S&P 500 
- Simple Moving Average
- Moving Average Convergence Divergence
- Adj Close Daily Return Histogram

In [ ]:
left, width = .25, .6
bottom, height = .25, .6
right = left + width
top = bottom + height

def simpleMovingAverage(values_df, window):
    return values_df.rolling(window=window).mean()

def expMovingAverage(values_df, window):
    return pd.Series.ewm(values_df, span=window, adjust=False).mean()


def movingAverageConvergenceDivergence(values_df, window_1 = 12, window_2 = 26, macd_signal_window=9):
    ema12 = expMovingAverage(values_df, window_1)
    ema26 = expMovingAverage(values_df, window_2)
    macd = ema12 - ema26
    macd_signal = expMovingAverage(macd, macd_signal_window)
    macd_hist = macd - macd_signal
    return macd, macd_signal, macd_hist

def plot_daily_return_hist(ticker):
    
    sp500_14_day_moving_average_df = simpleMovingAverage(sp500_adj_close_df, 14)
    sp500_50_day_moving_average_df = simpleMovingAverage(sp500_adj_close_df, 50)

    sp500_macd_df_window, sp500_macd_signal_df_window, sp500_macd_hist_df_window = movingAverageConvergenceDivergence(sp500_adj_close_df)
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(20, 16))
    ax1.set_title(f"Stock Price & SMA ({ticker})", fontsize=s[1])
    ax1.set_ylabel(f"Price", fontsize=s[2])
    ax1.plot(sp500_adj_close_df.index.values, sp500_adj_close_df[ticker], label='Adj. Close', linewidth=w[3], color=c[1])
    ax1.plot(sp500_14_day_moving_average_df.index.values, sp500_14_day_moving_average_df[ticker], label='14 Day SMA', linewidth=w[3], color=c[2])
    ax1.plot(sp500_50_day_moving_average_df.index.values, sp500_50_day_moving_average_df[ticker], label='50 Day SMA', linewidth=w[3], color=c[3])
    
    ax1.legend(loc='upper left', title = 'Tickers')
    ax1.grid(alpha=ga)
    
    
    ax2.set_title(f"MACD ({ticker})", fontsize=s[1])
    ax2.set_ylabel(f"Price", fontsize=s[2])
    ax2.plot(sp500_macd_df_window.index.values, sp500_macd_df_window[ticker], label='MACD', linewidth=w[3], color=c[2])
    ax2.plot(sp500_macd_signal_df_window.index.values, sp500_macd_signal_df_window[ticker], label='MACD Singnal', linewidth=w[3], color=c[3])
    
    ax2.legend(loc='upper left', title = 'Tickers')
    ax2.grid(alpha=ga)
    
    ax3.set_title(f"Daily Return ({ticker})", fontsize=s[1])
    ax3.set_ylabel(f"Frequency", fontsize=s[2])
    
    ax3.hist(dr[ticker], bins=50, label=ticker, linewidth=w[3], color=c[1])
    
    ax3.axvline(mdr.loc[ticker].values, color='red', linestyle='dashed', linewidth=2)
    
    # To plot the std lines we plot both the positive and negative values 
    ax3.axvline(sdr.loc[ticker].values, color='g', linestyle='dashed', linewidth=2)
    ax3.axvline(-sdr.loc[ticker].values, color='g', linestyle='dashed', linewidth=2)


    text = "Daily Returns\nMean: {}\nSTD: +/- {}\nSkew: {}\nKurtosis: {}".format(round(mdr.loc[ticker].values[0], 4), round(sdr.loc[ticker].values[0], 4), round(dr[ticker].skew(), 4), round(dr[ticker].kurtosis(), 4))
    ax3.text(right, top, text,
        fontsize='large',
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax3.transAxes)
    
    ax3.legend(loc='upper left', title = 'Tickers')
    ax3.grid(alpha=ga)
    
    plt.xticks(fontsize=s[3])
    plt.yticks(fontsize=s[3])
    plt.show()
    
    return 

interact(plot_daily_return_hist, 
         ticker=sp500_adj_close_df);




In [ ]:
dr_cov_matrix = (dr.cov())*252
dr_cov_matrix

In [ ]:
returns_fig = sns.PairGrid(sp500_adj_close_df[cr_highest_25])
returns_fig.map_upper(plt.scatter,color='purple')
returns_fig.map_lower(sns.kdeplot,cmap='cool_d')
returns_fig.map_diag(plt.hist,bins=50)

In [ ]:
returns_fig = sns.PairGrid(dr[mdr_highest_25].dropna())
returns_fig.map_upper(plt.scatter,color='purple')
returns_fig.map_lower(sns.kdeplot,cmap='cool_d')
returns_fig.map_diag(plt.hist,bins=50)

In [ ]:
# Correlation plot for the closing price
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(sp500_adj_close_df[cr_highest_25].corr(), annot=True, vmin=-1.0, vmax=1.0, cmap='RdBu',ax=ax)

In [ ]:
# Correlation plot for the daily returns
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(dr[mdr_highest_25].corr(), annot=True, vmin=-1.0, vmax=1.0, cmap='RdBu',ax=ax)

In [ ]:
rets = dr[mdr_highest_25].dropna()
plt.figure(figsize=(12, 10))
plt.scatter( rets.std(), rets.mean(), s=80)
plt.ylabel('Expected return')
plt.xlabel('Risk')
for label, x, y in zip(rets.columns, rets.std(), rets.mean()):
    plt.annotate(label, xy=(x, y), xytext=(50, 50), textcoords='offset points', ha='right', va='bottom',
    arrowprops=dict(arrowstyle='-', color='blue', connectionstyle='arc3,rad=-0.3'))

In [ ]:
# risk-free rate
rfr = 0
# Calculate the Sharpe ratio 
sr_s = (mdr['mean_daily_return'] - rfr)/sdr['std_daily_return']

sr_s.name = 'sharpe_ratio'
sr = sr_s.to_frame()

print('25 Highest Risk to Lower Risk')
sr_highest_25 = sr.sort_values(by='sharpe_ratio', ascending=False).head(25)
print(sr_highest_25)
print()
sr_highest_25.sharpe_ratio[0:5]

In [ ]:
# Calculate the Annualized Sharpe ratio 
ars_s = sr['sharpe_ratio'] * np.sqrt(252)
ars_s.name = 'annualized_sharpe_ratio'
sr = ars_s.to_frame()
sr

In [ ]:
left, width = .30 , .7
bottom, height = .25, .7
right = left + width
top = bottom + height


start_date = dt.datetime(1999, 1, 1)
end_date = dt.datetime(start_date.year + 1, 1, 1)
w = 4
h = 5
fig, ax = plt.subplots(w, h, figsize=(30, 20))

for i in range(w):
    for j in range(h):
        dr_year = daily_returns(sp500_adj_close_df.loc[start_date:end_date, :]).mean(1)[1:]
        dr_year.name = 'dr'
        dr_year = dr_year.to_frame()
        mdr_year = round(dr_year.mean().values[0], 4)
        sdr_year = round(dr_year.std().values[0], 4)
        skew_year = round(dr_year.dr.skew(), 4)
        kurtosis_year = round(dr_year.dr.kurtosis(), 4)
        
        
        ax[i][j].set_title(f"{str(start_date.year) + ' - ' + str(end_date.year)}", fontsize=s[1])
        ax[i][j].set_ylabel(f"Frequency", fontsize=s[2])
        ax[i][j].set_ylim(0, 50)
        ax[i][j].set_xlim(-12.0, 12.0)
        ax[i][j].hist(dr_year, bins=30)
        ax[i][j].axvline(mdr_year, color='red', linestyle='dashed', linewidth=2)

        # To plot the std lines we plot both the positive and negative values 
        ax[i][j].axvline(sdr_year, color='g', linestyle='dashed', linewidth=2)
        ax[i][j].axvline(-sdr_year, color='g', linestyle='dashed', linewidth=2)
        
 
        sr_s = (daily_returns_mean(sp500_adj_close_df.loc[start_date:end_date, :]) - rfr) / daily_returns_std(sp500_adj_close_df.loc[start_date:end_date, :]                                                                                                             )
        sr_s.name = 'sharpe_ratio'
        sr = sr_s.to_frame()   
        sr_highest_25 = sr.sort_values(by='sharpe_ratio', ascending=False).head(25)
        
        text1 = "\nDaily Returns\nMean: {}\nSTD: +/- {}\nSkew: {}\nKurtosis: {}".format(mdr_year, sdr_year, skew_year, kurtosis_year)
        text2 = "Top 5 Highest\nSharpe Ratio:\n{} - {}\n{} - {}\n{} - {}\n{} - {}\n{} - {}\n".format(
            sr_highest_25.index[0],  round(sr_highest_25.sharpe_ratio[0], 2),
            sr_highest_25.index[1],  round(sr_highest_25.sharpe_ratio[1], 2),
            sr_highest_25.index[2],  round(sr_highest_25.sharpe_ratio[2], 2),
            sr_highest_25.index[3],  round(sr_highest_25.sharpe_ratio[3], 2),
            sr_highest_25.index[4],  round(sr_highest_25.sharpe_ratio[4], 2))
        ax[i][j].text(right, top, text1,
            fontsize='large',
            horizontalalignment='right',
            verticalalignment='top',
            transform=ax[i][j].transAxes)
        ax[i][j].text(left, top, text2,
            fontsize='large',
            horizontalalignment='right',
            verticalalignment='top',
            transform=ax[i][j].transAxes)

              
        start_date = dt.datetime(start_date.year + 1, 1, 1)
        end_date = dt.datetime(end_date.year + 1, 1, 1)

plt.show()